In [1]:
import torch
import torch.nn as nn

# 시퀀스 설정
seq_len = 5
batch_size = 1
input_size = 3
hidden_size = 4

# 입력 텐서: (seq_len, batch_size, input_size)
x_seq = torch.randn(seq_len, batch_size, input_size)

In [2]:
x_seq

tensor([[[-0.2235, -0.0438, -1.5244]],

        [[ 2.6686,  1.2134, -0.1541]],

        [[ 0.3769, -0.8807,  0.7293]],

        [[-1.2839, -1.4658, -0.5082]],

        [[-0.6745, -0.9097,  2.0739]]])

In [3]:
x_seq.shape

torch.Size([5, 1, 3])

In [4]:
rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size)

# 초기 은닉 상태 (0으로 초기화)
h0 = torch.zeros(1, batch_size, hidden_size)

# 순전파
output, hn = rnn(x_seq, h0)

print(output.shape)  # torch.Size([seq_len, batch_size, hidden_size])
print(hn.shape)      # torch.Size([1, batch_size, hidden_size])

torch.Size([5, 1, 4])
torch.Size([1, 1, 4])


In [5]:
h0

tensor([[[0., 0., 0., 0.]]])

In [6]:
h0.shape

torch.Size([1, 1, 4])

In [7]:
lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)

# 초기 상태 설정 (은닉 상태 + 셀 상태)
h0 = torch.zeros(1, batch_size, hidden_size)
c0 = torch.zeros(1, batch_size, hidden_size)

# 순전파
output, (hn, cn) = lstm(x_seq, (h0, c0))

print(output.shape)  # (seq_len, batch_size, hidden_size)
print(hn.shape)      # (1, batch_size, hidden_size)
print(cn.shape)      # (1, batch_size, hidden_size)

torch.Size([5, 1, 4])
torch.Size([1, 1, 4])
torch.Size([1, 1, 4])
